# Tutorial 01: Document Loaders - Leitor de PDF Inteligente

**Objetivos de Aprendizagem:**
- Usar Document Loaders para carregar PDFs e outros documentos
- Implementar Text Splitters para dividir documentos em chunks
- Dominar estratégias de chunking para otimizar processamento
- Construir sistemas de análise de documentos com IA
- Criar sistemas de busca e filtros avançados

**Pré-requisitos:**
- Tutorial 01-LLMs-Basicos: Introdução aos LLMs concluído
- Tutorial 02-Prompts: ChatPromptTemplate (recomendado)
- Compreensão básica de Python e processamento de documentos

---

## Introdução

Este tutorial demonstra como usar Document Loaders e Text Splitters do LangChain para processar documentos PDF. Aprenderemos a carregar documentos, dividi-los em chunks otimizados e analisá-los com IA.

### O que são Document Loaders?

**Document Loaders** são componentes do LangChain que carregam documentos de diferentes fontes e os convertem em objetos `Document` padronizados.

**Tipos de Loaders:**
- PyPDFLoader: Carrega documentos PDF
- DirectoryLoader: Carrega múltiplos arquivos
- TextLoader: Carrega arquivos de texto
- WebBaseLoader: Carrega conteúdo de websites
- YouTubeLoader: Carrega transcrições do YouTube


### Conceitos-Chave

**Document Loaders:**
- PyPDFLoader: Carrega documentos PDF
- DirectoryLoader: Carrega múltiplos arquivos
- TextLoader: Carrega arquivos de texto
- WebBaseLoader: Carrega conteúdo de websites
- YouTubeLoader: Carrega transcrições do YouTube

**Text Splitters:**
- RecursiveCharacterTextSplitter: Divisor mais inteligente
- CharacterTextSplitter: Divisor por caracteres
- TokenTextSplitter: Divisor por tokens
- Estratégias de chunking otimizadas

**Estratégias de Chunking:**
- chunk_size: Tamanho ideal dos chunks (1000-2000 caracteres)
- chunk_overlap: Sobreposição entre chunks (200-400 caracteres)
- separators: Caracteres para dividir o texto
- length_function: Função para calcular tamanho

**Análise de Documentos com IA:**
- Resumo automático: Gerar resumos de chunks
- Extração de tópicos: Identificar temas principais
- Sistema de Q&A: Responder perguntas sobre o documento
- Análise de sentimento: Avaliar tom do conteúdo


### Casos de Uso

**Aplicações Reais:**
- Sistema de análise de contratos e documentos legais
- Plataforma de pesquisa acadêmica e científica
- Sistema de suporte com base de conhecimento
- Análise de relatórios financeiros e empresariais
- Sistema de busca em documentos corporativos

**Habilidades Desenvolvidas:**
- Processamento de documentos em larga escala
- Otimização de chunks para melhor performance
- Integração com IA para análise inteligente
- Sistemas de busca e filtros avançados
- Exportação de dados em múltiplos formatos

In [ ]:
# Configuração inicial
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Verificar se OPENAI_API_KEY está configurada
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️ ATENÇÃO: OPENAI_API_KEY não encontrada no .env")
    print("   Configure a chave antes de continuar.")
else:
    print("✅ Variáveis de ambiente carregadas com sucesso!")

In [ ]:
# Verificar se existe um PDF para teste
pdf_path = Path("assets/contrato.pdf")
if pdf_path.exists():
    print(f"✅ PDF encontrado: {pdf_path}")
else:
    print(f"⚠️ PDF não encontrado: {pdf_path}")
    print("   Certifique-se de ter um PDF na pasta assets/ para testar")

✅ API Key carregada com sucesso!
✅ PDF encontrado: contrato.pdf


## Exemplo 1: Carregar PDF com PyPDFLoader

Vamos carregar um documento PDF usando PyPDFLoader. O loader converte cada página do PDF em um objeto `Document` do LangChain.

**Estrutura do Document:**
- `page_content`: Texto extraído da página
- `metadata`: Metadados (source, page, etc.)

In [ ]:
# Função para carregar PDF
def carregar_pdf(caminho_pdf):
    """
    Carrega um PDF e retorna os documentos.
    
    Args:
        caminho_pdf: Caminho para o arquivo PDF
    
    Returns:
        Lista de objetos Document do LangChain
    """
    try:
        loader = PyPDFLoader(str(caminho_pdf))
        documentos = loader.load()
        print(f"✅ PDF carregado: {len(documentos)} páginas")
        return documentos
    except FileNotFoundError:
        print(f"⚠️ Arquivo {caminho_pdf} não encontrado")
        # Criar documento de exemplo
        from langchain_core.documents import Document
        documento_exemplo = Document(
            page_content="Este é um contrato de exemplo. Contém cláusulas importantes sobre serviços, pagamentos e responsabilidades. As partes concordam em cumprir todos os termos estabelecidos.",
            metadata={"source": "exemplo", "page": 1}
        )
        print("📄 Documento de exemplo criado")
        return [documento_exemplo]

# Testar o loader (se PDF existir)
if pdf_path.exists():
    documentos = carregar_pdf(pdf_path)
    print(f"\nPrimeira página (primeiros 200 caracteres):")
    print(documentos[0].page_content[:200] + "...")
    print(f"\nMetadados da primeira página:")
    print(documentos[0].metadata)
documentos = carregar_pdf(pdf_path)
print(f"📚 Documentos carregados: {len(documentos)}")

incorrect startxref pointer(1)
parsing for Object Streams


✅ PDF carregado: 4 páginas
📚 Documentos carregados: 4


## Por que dividir textos?

### Problemas com textos longos:
- Limite de tokens dos LLMs
- Perda de contexto específico
- Dificuldade para análise focada

### Solução: Chunking inteligente
- Divide em pedaços menores
- Mantém sobreposição para contexto
- Preserva estrutura semântica

In [4]:
# 5. CONFIGURAR TEXT SPLITTER
def configurar_text_splitter(chunk_size=1000, chunk_overlap=200):
    """
    🧠 O que faz: Divide textos grandes em pedaços menores (chunks)
    
    Por que precisamos disso?
    - LLMs têm limite de tokens
    - Chunks menores = melhor compreensão
    - Overlap = mantém contexto entre pedaços
    
    Parâmetros:
    - chunk_size: Tamanho máximo de cada pedaço
    - chunk_overlap: Sobreposição para manter contexto
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]  # Onde quebrar o texto
    )
    return text_splitter

# Testar o text splitter
text_splitter = configurar_text_splitter()
print("✂️ Text splitter configurado!")
print(f"   - Chunk size: {text_splitter._chunk_size}")
print(f"   - Chunk overlap: {text_splitter._chunk_overlap}")


✂️ Text splitter configurado!
   - Chunk size: 1000
   - Chunk overlap: 200


In [10]:
# 6. DIVIDIR DOCUMENTOS EM CHUNKS
def dividir_documentos(documentos, text_splitter):
    """
    📄 O que faz: Pega os documentos e divide em pedaços menores
    
    Por que isso é importante?
    - Cada chunk pode ser processado separadamente
    - Facilita a busca e recuperação de informações
    - Permite processar documentos muito grandes
    """
    chunks = []
    for doc in documentos:
        doc_chunks = text_splitter.split_documents([doc])
        chunks.extend(doc_chunks)
    
    print(f"�� Documento dividido em {len(chunks)} chunks")
    for i, chunk in enumerate(chunks[:3]):  # Mostrar primeiros 3 chunks
        print(f"\n--- Chunk {i+1} ---")
        print(f"Tamanho: {len(chunk.page_content)} caracteres")
        print(f"Conteúdo: {chunk.page_content[:100]}...")
    
    return chunks

# Dividir documentos
chunks = dividir_documentos(documentos, text_splitter)


�� Documento dividido em 7 chunks

--- Chunk 1 ---
Tamanho: 941 caracteres
Conteúdo: CONTRATO DE PRESTAÇÃO DE SERVIÇOS
 TECNOLÓGICOS
CONTRATANTE:
TechCorp Solutions Ltda., pessoa jurídi...

--- Chunk 2 ---
Tamanho: 723 caracteres
Conteúdo: e inscrita no CPF sob o nº 987.654.321-00.
OBJETO DO CONTRATO:
O presente contrato tem por objeto a ...

--- Chunk 3 ---
Tamanho: 940 caracteres
Conteúdo: PRAZO DE EXECUÇÃO:
O prazo para execução total dos serviços será de 180 (cento e oitenta) dias corri...


## Estratégia de Análise com IA

### Abordagem em camadas:
1. **Resumo de cada chunk**: Captura ideias principais
2. **Identificação de tópicos**: Extrai temas e conceitos
3. **Sistema de Q&A**: Permite perguntas específicas

### Vantagens:
- Análise estruturada
- Foco em informações relevantes
- Facilita compreensão de documentos longos

In [11]:
# Configurar modelo de IA
chat_model = ChatOpenAI(
    temperature=0.1,  # Baixa temperatura para respostas consistentes
    model_name="gpt-3.5-turbo"
)

# Template para resumo
template_resumo = ChatPromptTemplate.from_messages([
    ("system", "Você é um especialista em análise de documentos. Forneça resumos concisos e objetivos."),
    ("human", "Resuma o seguinte texto em 2-3 frases:\n\n{texto}")
])

# Template para análise de tópicos
template_topicos = ChatPromptTemplate.from_messages([
    ("system", "Identifique os 3 principais tópicos deste texto."),
    ("human", "Texto:\n\n{texto}")
])

# Criar chains
chain_resumo = template_resumo | chat_model | StrOutputParser()
chain_topicos = template_topicos | chat_model | StrOutputParser()

print("�� Sistema de análise configurado!")

�� Sistema de análise configurado!


In [ ]:
def analisar_chunks(chunks):
    """Analisa cada chunk com IA"""
    resultados = []
    
    for i, chunk in enumerate(chunks):
        print(f"\n�� Analisando Chunk {i+1}...")
        
        # Resumo
        resumo = chain_resumo.invoke({"texto": chunk.page_content})
        print(f"📝 Resumo: {resumo}")
        
        # Tópicos
        topicos = chain_topicos.invoke({"texto": chunk.page_content})
        print(f"��️ Tópicos: {topicos}")
        
        resultados.append({
            "chunk_id": i+1,
            "resumo": resumo,
            "topicos": topicos,
            "tamanho": len(chunk.page_content)
        })
    
    return resultados

# Executar análise
resultados_analise = analisar_chunks(chunks)


�� Analisando Chunk 1...
📝 Resumo: O contrato de prestação de serviços tecnológicos é entre a TechCorp Solutions Ltda. e a Inovação Digital Ltda., com o objetivo de desenvolvimento de serviços tecnológicos. Os representantes legais de ambas as partes são João Silva Santos e Maria Oliveira Costa, respectivamente.
��️ Tópicos: 1. Contrato de prestação de serviços tecnológicos
2. Partes envolvidas no contrato (Contratante e Contratada)
3. Objeto do contrato (serviços de desenvolvimento tecnológico)

�� Analisando Chunk 2...
📝 Resumo: O contrato tem como objetivo a prestação de serviços de desenvolvimento de software personalizado para gestão empresarial, utilizando tecnologias como Python, JavaScript, Django, React, PostgreSQL e AWS. O sistema incluirá módulos para gestão de clientes, controle de estoque, faturamento, relatórios gerenciais e integração com sistemas externos.
��️ Tópicos: 1. Objeto do contrato: prestação de serviços de desenvolvimento de software personalizado.
2. Especif

In [13]:
# Template para Q&A
template_qa = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especialista em documentos. Responda baseado apenas no contexto fornecido."),
    ("human", "Contexto:\n{contexto}\n\nPergunta: {pergunta}")
])

# Chain para Q&A
chain_qa = template_qa | chat_model | StrOutputParser()

def responder_pergunta(pergunta, chunks):
    """Responde perguntas baseado no conteúdo dos chunks"""
    # Concatenar conteúdo relevante
    contexto = "\n\n".join([chunk.page_content for chunk in chunks])
    
    # Gerar resposta
    resposta = chain_qa.invoke({
        "contexto": contexto,
        "pergunta": pergunta
    })
    
    return resposta

# Testar sistema de Q&A
perguntas_teste = [
    "Quais são os principais tópicos deste documento?",
    "Há alguma cláusula importante mencionada?",
    "Qual é o objetivo geral deste documento?"
]

print("❓ Testando sistema de perguntas e respostas:")
for pergunta in perguntas_teste:
    print(f"\n❓ Pergunta: {pergunta}")
    resposta = responder_pergunta(pergunta, chunks)
    print(f"💡 Resposta: {resposta}")

❓ Testando sistema de perguntas e respostas:

❓ Pergunta: Quais são os principais tópicos deste documento?
💡 Resposta: Os principais tópicos deste documento são:

1. Identificação das partes envolvidas (Contratante e Contratada).
2. Objeto do contrato: prestação de serviços de desenvolvimento de software personalizado.
3. Especificações técnicas do sistema a ser desenvolvido.
4. Prazo de execução dos serviços.
5. Valor e forma de pagamento.
6. Responsabilidades da Contratada.
7. Responsabilidades da Contratante.
8. Confidencialidade das informações trocadas durante a execução do contrato.
9. Propriedade intelectual dos materiais desenvolvidos.
10. Rescisão do contrato.
11. Foro para resolução de dúvidas ou litígios.
12. Assinatura das partes e testemunhas.

❓ Pergunta: Há alguma cláusula importante mencionada?
💡 Resposta: Sim, há várias cláusulas importantes mencionadas no contrato de prestação de serviços tecnológicos, tais como:

1. Objeto do contrato: Desenvolvimento de software per

## 🎯 Desafios para Praticar

### Desafio 1: Diferentes Tipos de PDF
- Teste com PDFs de diferentes tamanhos
- Compare resultados com contratos, relatórios, artigos


In [ ]:
def comparar_configuracoes_chunks(documentos):
    """Compara diferentes configurações de chunking"""
    configs = [
        (500, 50),   # Chunks pequenos, pouca sobreposição
        (1000, 200), # Configuração padrão
        (2000, 400), # Chunks grandes, muita sobreposição
        (500, 200),  # Chunks pequenos, muita sobreposição
    ]
    
    resultados = []
    
    for chunk_size, chunk_overlap in configs:
        print(f"\n🔧 Testando: chunk_size={chunk_size}, chunk_overlap={chunk_overlap}")
        
        # Configurar splitter
        splitter = configurar_text_splitter(chunk_size, chunk_overlap)
        
        # Dividir documentos
        chunks = dividir_documentos(documentos, splitter)
        
        # Análise rápida
        resumo_geral = chain_resumo.invoke({
            "texto": "\n".join([chunk.page_content for chunk in chunks])
        })
        
        resultados.append({
            "config": (chunk_size, chunk_overlap),
            "num_chunks": len(chunks),
            "resumo": resumo_geral
        })
        
        print(f"   📊 Número de chunks: {len(chunks)}")
        print(f"   📝 Resumo: {resumo_geral}")
    
    return resultados

# Executar comparação
resultados_comparacao = comparar_configuracoes_chunks(documentos)


🔧 Testando: chunk_size=500, chunk_overlap=50
�� Documento dividido em 13 chunks

--- Chunk 1 ---
Tamanho: 451 caracteres
Conteúdo: CONTRATO DE PRESTAÇÃO DE SERVIÇOS
 TECNOLÓGICOS
CONTRATANTE:
TechCorp Solutions Ltda., pessoa jurídi...

--- Chunk 2 ---
Tamanho: 420 caracteres
Conteúdo: CONTRATADA:
Inovação Digital Ltda., pessoa jurídica de direito privado, inscrita no CNPJ sob o nº
98...

--- Chunk 3 ---
Tamanho: 453 caracteres
Conteúdo: OBJETO DO CONTRATO:
O presente contrato tem por objeto a prestação de serviços de desenvolvimento de...
   📊 Número de chunks: 13
   📝 Resumo: O contrato de prestação de serviços tecnológicos entre TechCorp Solutions Ltda. e Inovação Digital Ltda. tem como objeto o desenvolvimento de um sistema de gestão empresarial, com prazo de execução de 180 dias e valor total de R$ 150.000,00, dividido em três parcelas. Ambas as partes se comprometem a manter a confidencialidade das informações trocadas, com a propriedade intelectual do projeto pertencendo exclusi


### Desafio 2: Otimização de Chunks
- Teste diferentes valores de chunk_size e chunk_overlap
- Encontre a configuração ideal para seu tipo de documento


In [ ]:
import json
import csv
from datetime import datetime

def exportar_resultados(resultados_analise, resultados_comparacao, filename_prefix="analise_pdf"):
    """Exporta resultados em diferentes formatos"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Preparar dados para exportação
    dados_export = {
        "timestamp": timestamp,
        "analise_chunks": resultados_analise,
        "comparacao_configs": resultados_comparacao
    }
    
    # 1. JSON
    json_filename = f"{filename_prefix}_{timestamp}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(dados_export, f, ensure_ascii=False, indent=2)
    print(f"💾 JSON salvo: {json_filename}")
    
    # 2. CSV para análise de chunks
    csv_filename = f"{filename_prefix}_chunks_{timestamp}.csv"
    with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['chunk_id', 'resumo', 'topicos', 'tamanho'])
        writer.writeheader()
        for resultado in resultados_analise:
            writer.writerow(resultado)
    print(f"💾 CSV salvo: {csv_filename}")
    
    # 3. Markdown
    md_filename = f"{filename_prefix}_{timestamp}.md"
    with open(md_filename, 'w', encoding='utf-8') as f:
        f.write(f"# Análise de PDF - {timestamp}\n\n")
        
        f.write("## Análise por Chunks\n\n")
        for resultado in resultados_analise:
            f.write(f"### Chunk {resultado['chunk_id']}\n")
            f.write(f"- **Resumo**: {resultado['resumo']}\n")
            f.write(f"- **Tópicos**: {resultado['topicos']}\n")
            f.write(f"- **Tamanho**: {resultado['tamanho']} caracteres\n\n")
        
        f.write("## Comparação de Configurações\n\n")
        for resultado in resultados_comparacao:
            chunk_size, chunk_overlap = resultado['config']
            f.write(f"### Configuração: {chunk_size}/{chunk_overlap}\n")
            f.write(f"- **Chunks**: {resultado['num_chunks']}\n")
            f.write(f"- **Resumo**: {resultado['resumo']}\n\n")
    
    print(f"💾 Markdown salvo: {md_filename}")
    print("✅ Exportação concluída em 3 formatos!")

# Executar exportação
exportar_resultados(resultados_analise, resultados_comparacao)

💾 JSON salvo: analise_pdf_20250904_181253.json
💾 CSV salvo: analise_pdf_chunks_20250904_181253.csv
💾 Markdown salvo: analise_pdf_20250904_181253.md
✅ Exportação concluída em 3 formatos!



### Desafio 3: Sistema de Busca
- Implemente busca por palavras-chave nos chunks
- Crie filtros por metadados


In [22]:
# Desafio 3: Sistema de Busca
def buscar_palavras_chave(chunks, palavra_chave, metadados_filtro=None):
    """
    Busca por palavras-chave nos chunks com filtros opcionais
    
    Args:
        chunks: Lista de chunks para buscar
        palavra_chave: Palavra-chave para buscar
        metadados_filtro: Dicionário com filtros de metadados (ex: {'source': 'arquivo.pdf'})
    
    Returns:
        Lista de chunks que contêm a palavra-chave
    """
    resultados = []
    
    for chunk in chunks:
        # Verifica se a palavra-chave está no conteúdo
        if palavra_chave.lower() in chunk.page_content.lower():
            # Aplica filtros de metadados se fornecidos
            if metadados_filtro:
                chunk_metadados = chunk.metadata
                filtro_ok = True
                
                for chave, valor in metadados_filtro.items():
                    if chave not in chunk_metadados or chunk_metadados[chave] != valor:
                        filtro_ok = False
                        break
                
                if filtro_ok:
                    resultados.append(chunk)
            else:
                resultados.append(chunk)
    
    return resultados

def buscar_por_topicos(chunks, topico):
    """
    Busca chunks relacionados a um tópico específico
    
    Args:
        chunks: Lista de chunks para buscar
        topico: Tópico para buscar
    
    Returns:
        Lista de chunks relacionados ao tópico
    """
    output_parser = StrOutputParser()
    
    # Cria um prompt para identificar se o chunk é relacionado ao tópico
    prompt_topicos = ChatPromptTemplate.from_messages([
        ("system", "Você é um especialista em análise de conteúdo. Determine se o texto fornecido está relacionado ao tópico especificado. Responda apenas 'SIM' ou 'NÃO'."),
        ("human", "Tópico: {topico}\n\nTexto: {texto}")
    ])
    
    chain_topicos = prompt_topicos | chat_model | output_parser
    resultados = []
    
    for chunk in chunks:
        resposta = chain_topicos.invoke({"topico": topico, "texto": chunk.page_content})
        if resposta.upper() == "SIM":
            resultados.append(chunk)
    
    return resultados

def criar_indice_busca(chunks):
    """
    Cria um índice de busca para facilitar consultas
    
    Args:
        chunks: Lista de chunks para indexar
    
    Returns:
        Dicionário com índice de busca
    """
    indice = {
        'por_palavra': {},
        'por_tamanho': {},
        'por_fonte': {},
        'por_pagina': {}
    }
    
    for i, chunk in enumerate(chunks):
        # Índice por palavra-chave
        palavras = chunk.page_content.lower().split()
        for palavra in palavras:
            if palavra not in indice['por_palavra']:
                indice['por_palavra'][palavra] = []
            indice['por_palavra'][palavra].append(i)
        
        # Índice por tamanho
        tamanho = len(chunk.page_content)
        if tamanho not in indice['por_tamanho']:
            indice['por_tamanho'][tamanho] = []
        indice['por_tamanho'][tamanho].append(i)
        
        # Índice por fonte
        fonte = chunk.metadata.get('source', 'desconhecida')
        if fonte not in indice['por_fonte']:
            indice['por_fonte'][fonte] = []
        indice['por_fonte'][fonte].append(i)
        
        # Índice por página
        pagina = chunk.metadata.get('page', 0)
        if pagina not in indice['por_pagina']:
            indice['por_pagina'][pagina] = []
        indice['por_pagina'][pagina].append(i)
    
    return indice

def buscar_com_indice(indice, chunks, palavra_chave=None, tamanho_min=None, fonte=None, pagina=None):
    """
    Busca usando o índice criado
    
    Args:
        indice: Índice de busca
        chunks: Lista de chunks
        palavra_chave: Palavra-chave para buscar
        tamanho_min: Tamanho mínimo do chunk
        fonte: Fonte específica
        pagina: Página específica
    
    Returns:
        Lista de chunks que atendem aos critérios
    """
    indices_encontrados = set(range(len(chunks)))
    
    # Filtro por palavra-chave
    if palavra_chave:
        palavra_lower = palavra_chave.lower()
        if palavra_lower in indice['por_palavra']:
            indices_encontrados = indices_encontrados.intersection(set(indice['por_palavra'][palavra_lower]))
        else:
            return []  # Palavra não encontrada
    
    # Filtro por tamanho mínimo
    if tamanho_min:
        indices_tamanho = []
        for tamanho, indices in indice['por_tamanho'].items():
            if tamanho >= tamanho_min:
                indices_tamanho.extend(indices)
        indices_encontrados = indices_encontrados.intersection(set(indices_tamanho))
    
    # Filtro por fonte
    if fonte:
        if fonte in indice['por_fonte']:
            indices_encontrados = indices_encontrados.intersection(set(indice['por_fonte'][fonte]))
        else:
            return []  # Fonte não encontrada
    
    # Filtro por página
    if pagina is not None:
        if pagina in indice['por_pagina']:
            indices_encontrados = indices_encontrados.intersection(set(indice['por_pagina'][pagina]))
        else:
            return []  # Página não encontrada
    
    return [chunks[i] for i in indices_encontrados]

# Teste do sistema de busca
print("🔍 TESTANDO SISTEMA DE BUSCA")
print("=" * 50)

# Carrega o PDF de exemplo
documento = carregar_pdf("contrato.pdf")
text_splitter = configurar_text_splitter(chunk_size=1000, chunk_overlap=200)
chunks = dividir_documentos(documento, text_splitter)

# Cria índice de busca
indice = criar_indice_busca(chunks)
print(f"✅ Índice criado com {len(chunks)} chunks")

# Busca por palavra-chave
resultados_palavra = buscar_palavras_chave(chunks, "inteligência")
print(f"🔍 Encontrados {len(resultados_palavra)} chunks com 'inteligência'")

# Busca por tópico
resultados_topico = buscar_por_topicos(chunks, "machine learning")
print(f"📚 Encontrados {len(resultados_topico)} chunks sobre 'machine learning'")

# Busca com filtros
resultados_filtrados = buscar_com_indice(indice, chunks, palavra_chave="dados", tamanho_min=500)
print(f"🎯 Encontrados {len(resultados_filtrados)} chunks com 'dados' e tamanho >= 500")

print("\n✅ Sistema de busca implementado com sucesso!")

incorrect startxref pointer(1)
parsing for Object Streams


🔍 TESTANDO SISTEMA DE BUSCA
✅ PDF carregado: 4 páginas
�� Documento dividido em 7 chunks

--- Chunk 1 ---
Tamanho: 941 caracteres
Conteúdo: CONTRATO DE PRESTAÇÃO DE SERVIÇOS
 TECNOLÓGICOS
CONTRATANTE:
TechCorp Solutions Ltda., pessoa jurídi...

--- Chunk 2 ---
Tamanho: 723 caracteres
Conteúdo: e inscrita no CPF sob o nº 987.654.321-00.
OBJETO DO CONTRATO:
O presente contrato tem por objeto a ...

--- Chunk 3 ---
Tamanho: 940 caracteres
Conteúdo: PRAZO DE EXECUÇÃO:
O prazo para execução total dos serviços será de 180 (cento e oitenta) dias corri...
✅ Índice criado com 7 chunks
🔍 Encontrados 0 chunks com 'inteligência'
📚 Encontrados 0 chunks sobre 'machine learning'
🎯 Encontrados 2 chunks com 'dados' e tamanho >= 500

✅ Sistema de busca implementado com sucesso!



### Desafio 4: Exportação de Resultados
- Salve análises em diferentes formatos (JSON, CSV, Markdown)
- Crie relatórios estruturados

In [23]:
import json
import csv
from datetime import datetime
import os

def exportar_analise_json(resultados_analise, filename="analise_pdf.json"):
    """
    Exporta resultados da análise em formato JSON
    
    Args:
        resultados_analise: Dicionário com resultados da análise
        filename: Nome do arquivo para salvar
    """
    # Prepara dados para exportação
    dados_export = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "total_chunks": len(resultados_analise),
            "versao": "1.0"
        },
        "resultados": []
    }
    
    for i, resultado in enumerate(resultados_analise):
        dados_export["resultados"].append({
            "chunk_id": i,
            "resumo": resultado["resumo"],
            "topicos": resultado["topicos"],
            "tamanho": resultado["tamanho"],
            "pagina": resultado["pagina"],
            "fonte": resultado["fonte"]
        })
    
    # Salva arquivo JSON
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dados_export, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Análise exportada para {filename}")

def exportar_analise_csv(resultados_analise, filename="analise_pdf.csv"):
    """
    Exporta resultados da análise em formato CSV
    
    Args:
        resultados_analise: Dicionário com resultados da análise
        filename: Nome do arquivo para salvar
    """
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        
        # Cabeçalho
        writer.writerow(['Chunk ID', 'Resumo', 'Tópicos', 'Tamanho', 'Página', 'Fonte'])
        
        # Dados
        for i, resultado in enumerate(resultados_analise):
            writer.writerow([
                i,
                resultado["resumo"],
                ", ".join(resultado["topicos"]),
                resultado["tamanho"],
                resultado["pagina"],
                resultado["fonte"]
            ])
    
    print(f"✅ Análise exportada para {filename}")

def exportar_analise_markdown(resultados_analise, filename="analise_pdf.md"):
    """
    Exporta resultados da análise em formato Markdown
    
    Args:
        resultados_analise: Dicionário com resultados da análise
        filename: Nome do arquivo para salvar
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("# Análise de PDF com IA\n\n")
        f.write(f"**Data:** {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
        f.write(f"**Total de Chunks:** {len(resultados_analise)}\n\n")
        
        f.write("## Resumo Executivo\n\n")
        f.write("Este documento contém a análise automática de um PDF utilizando técnicas de IA.\n\n")
        
        f.write("## Resultados por Chunk\n\n")
        
        for i, resultado in enumerate(resultados_analise):
            f.write(f"### Chunk {i+1}\n\n")
            f.write(f"**Página:** {resultado['pagina']}\n")
            f.write(f"**Fonte:** {resultado['fonte']}\n")
            f.write(f"**Tamanho:** {resultado['tamanho']} caracteres\n\n")
            
            f.write("**Resumo:**\n")
            f.write(f"{resultado['resumo']}\n\n")
            
            f.write("**Tópicos Identificados:**\n")
            for topico in resultado['topicos']:
                f.write(f"- {topico}\n")
            f.write("\n---\n\n")
    
    print(f"✅ Análise exportada para {filename}")

def exportar_relatorio_completo(resultados_analise, resultados_busca=None, filename_prefix="relatorio_pdf"):
    """
    Exporta relatório completo em múltiplos formatos
    
    Args:
        resultados_analise: Resultados da análise
        resultados_busca: Resultados de buscas (opcional)
        filename_prefix: Prefixo para os arquivos
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Exporta em diferentes formatos
    exportar_analise_json(resultados_analise, f"{filename_prefix}_{timestamp}.json")
    exportar_analise_csv(resultados_analise, f"{filename_prefix}_{timestamp}.csv")
    exportar_analise_markdown(resultados_analise, f"{filename_prefix}_{timestamp}.md")
    
    # Cria relatório executivo
    relatorio_executivo = f"{filename_prefix}_executivo_{timestamp}.md"
    with open(relatorio_executivo, 'w', encoding='utf-8') as f:
        f.write("# Relatório Executivo - Análise de PDF\n\n")
        f.write(f"**Data:** {datetime.now().strftime('%d/%m/%Y %H:%M')}\n\n")
        
        f.write("## Estatísticas Gerais\n\n")
        f.write(f"- **Total de Chunks:** {len(resultados_analise)}\n")
        f.write(f"- **Páginas Analisadas:** {len(set(r['pagina'] for r in resultados_analise))}\n")
        f.write(f"- **Tamanho Médio dos Chunks:** {sum(r['tamanho'] for r in resultados_analise) // len(resultados_analise)} caracteres\n\n")
        
        f.write("## Tópicos Mais Frequentes\n\n")
        todos_topicos = []
        for resultado in resultados_analise:
            todos_topicos.extend(resultado['topicos'])
        
        from collections import Counter
        topicos_frequentes = Counter(todos_topicos).most_common(10)
        
        for topico, frequencia in topicos_frequentes:
            f.write(f"- **{topico}:** {frequencia} ocorrências\n")
        
        f.write("\n## Recomendações\n\n")
        f.write("1. **Foco em Tópicos Principais:** Concentre-se nos tópicos mais frequentes\n")
        f.write("2. **Análise Detalhada:** Use os chunks maiores para análise aprofundada\n")
        f.write("3. **Busca Estratégica:** Utilize o sistema de busca para encontrar informações específicas\n")
        f.write("4. **Monitoramento:** Acompanhe a evolução dos tópicos ao longo do documento\n")
    
    print(f"✅ Relatório executivo exportado para {relatorio_executivo}")
    print(f"✅ Relatório completo exportado com prefixo: {filename_prefix}_{timestamp}")

def exportar_resultados_busca(resultados_busca, filename="resultados_busca.json"):
    """
    Exporta resultados de busca em formato JSON
    
    Args:
        resultados_busca: Dicionário com resultados de busca
        filename: Nome do arquivo para salvar
    """
    dados_export = {
        "metadata": {
            "timestamp": datetime.now().isoformat(),
            "total_resultados": len(resultados_busca),
            "versao": "1.0"
        },
        "resultados": []
    }
    
    for resultado in resultados_busca:
        dados_export["resultados"].append({
            "conteudo": resultado.page_content,
            "metadados": resultado.metadata,
            "tamanho": len(resultado.page_content)
        })
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dados_export, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Resultados de busca exportados para {filename}")

# Teste da exportação
print("📊 TESTANDO EXPORTAÇÃO DE RESULTADOS")
print("=" * 50)

# Simula resultados de análise para teste
resultados_teste = [
    {
        "resumo": "Introdução aos conceitos de IA",
        "topicos": ["inteligência artificial", "machine learning", "introdução"],
        "tamanho": 500,
        "pagina": 1,
        "fonte": "exemplo.pdf"
    },
    {
        "resumo": "Aplicações práticas de ML",
        "topicos": ["aplicações", "prática", "machine learning"],
        "tamanho": 750,
        "pagina": 2,
        "fonte": "exemplo.pdf"
    }
]

# Exporta em diferentes formatos
exportar_analise_json(resultados_teste, "teste_analise.json")
exportar_analise_csv(resultados_teste, "teste_analise.csv")
exportar_analise_markdown(resultados_teste, "teste_analise.md")
exportar_relatorio_completo(resultados_teste, filename_prefix="teste_relatorio")

print("\n✅ Sistema de exportação implementado com sucesso!")
print("📁 Verifique os arquivos gerados no diretório atual")

📊 TESTANDO EXPORTAÇÃO DE RESULTADOS
✅ Análise exportada para teste_analise.json
✅ Análise exportada para teste_analise.csv
✅ Análise exportada para teste_analise.md
✅ Análise exportada para teste_relatorio_20250904_190726.json
✅ Análise exportada para teste_relatorio_20250904_190726.csv
✅ Análise exportada para teste_relatorio_20250904_190726.md
✅ Relatório executivo exportado para teste_relatorio_executivo_20250904_190726.md
✅ Relatório completo exportado com prefixo: teste_relatorio_20250904_190726

✅ Sistema de exportação implementado com sucesso!
📁 Verifique os arquivos gerados no diretório atual
